<a href="https://colab.research.google.com/github/prakashsukhwal/NLP-common-notebooks/blob/main/Scrape_results_from_google_search_SERPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scraping google search (using keywords)




In [ ]:
# required package and initial settings for reading a data file from google drive in google colab
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

# other required packages depending on the need
import pandas as pd
import re
from tqdm import tqdm
!pip3 install --upgrade ecommercetools
from ecommercetools import seo

## Step2: get input data on what to query or search in google

In [ ]:
worksheet1 = gc.open('search keywords').worksheet('Sheet1')
# get_all_values gives a list of rows.
rows = worksheet1.get_all_values()
# Convert to a DataFrame and render.
data = pd.DataFrame.from_records(rows[1:],columns=rows[0])

In [ ]:
data.head()

,location,role,topic keyword
0,denmark,police chief,inflation
1,denmark,mayor,inflation
2,denmark,governer,inflation
3,denmark,police chief,immigration
4,denmark,mayor,immigration


## Step3: create your query list of keywords to be used in google search based on input data

In [ ]:
df1 = data.drop_duplicates(['location','topic keyword'])
Querylist1 = (df1['location'] + ', ' + df1['topic keyword']).tolist()
df1 = data.drop_duplicates(['location','role'])
Querylist2 = (df1['location'] + ', ' + df1['role']).tolist()

In [ ]:
Querylist1

['denmark, inflation',
 'denmark, immigration',
 'denmark, global warming',
 'toronto, inflation',
 'toronto, immigration',
 'toronto, global warming',
 'washington, inflation',
 'washington, immigration',
 'washington, global warming',
 'madrid, inflation',
 'madrid, immigration',
 'madrid, global warming',
 'paris, inflation',
 'paris, immigration',
 'paris, global warming',
 'sweden, inflation',
 'sweden, immigration',
 'sweden, global warming']

# Step4: get the query data from google and filter based on a given conditions and store it

In [ ]:
# def get_google_links(query, pages_num, condition):
targets = []
for query in tqdm(Querylist1):
  # step 4.1: specify how many pages of google search you want to search!
  pages_num = 5
  # step 4.2: specify what type of google search results you want to grab 
  # and focus given this list of keywords
  conditions = ['wikipedia','twitter','linkedin','facebook','news'] # use [''] to get all the google search results
  # Step 4.3: the main body. 
  try:
    res = seo.get_serps(query, pages=pages_num)
  except:
    continue
  for c in conditions[:]:
    temp1 = res[res['link'].str.contains(c,case=False)]['title'].to_list()
    temp2 = res[res['link'].str.contains(c,case=False)]['link'].to_list()
    if len(temp1)==0:
      y = ''
    else:
      y = temp1[0]
    if len(temp2)==0:
      z = ''
      continue
    else:
      z = temp2[0]
    targets.append([y,z,c,query])

100%|██████████| 18/18 [01:02<00:00,  3.45s/it]


In [ ]:
#conver the result to a dataframe with given column names
targets = pd.DataFrame(targets, columns=['title','link','link category','query keyword used'])

In [ ]:
targets

,title,link,link category,query keyword used
0,Economy of Denmark - Wikipedia,https://en.wikipedia.org/wiki/Economy_of_Denmark,wikipedia,"denmark, inflation"
1,"Denmark Inflation Hits New High as Food, Energ...",https://www.bloomberg.com/news/articles/2022-0...,news,"denmark, inflation"
2,Immigration to Denmark - Wikipedia,https://en.wikipedia.org/wiki/Immigration_to_D...,wikipedia,"denmark, immigration"
3,Denmark Plans to Tighten Deportation Rules for...,https://www.schengenvisainfo.com/news/denmark-...,news,"denmark, immigration"
4,Why Denmark Wants to Be a 'Frontrunner' in the...,https://environment.yale.edu/news/article/why-...,news,"denmark, global warming"
5,Canadian property bubble - Wikipedia,https://en.wikipedia.org/wiki/Canadian_propert...,wikipedia,"toronto, inflation"
6,U of T economist unpacks soaring inflation,https://www.utoronto.ca/news/u-t-economist-unp...,news,"toronto, inflation"
7,KJ Toronto Immigration Inc - Home | Facebook,https://www.facebook.com/KJTORONTOIMMIGRATION/,facebook,"toronto, immigration"
8,Immigration | News | The Star,https://www.thestar.com/news/immigration.html,news,"toronto, immigration"
9,Toronto Conference on the Changing Atmosphere ...,https://en.wikipedia.org/wiki/Toronto_Conferen...,wikipedia,"toronto, global warming"


# Store the result

In [ ]:
# Open our new sheet and add some data.
sheet = gc.open('search keywords')
sheet.add_worksheet('google_search_results',rows=250, cols=20) 
worksheet3 = gc.open('search keywords').worksheet('google_search_results')
worksheet3.update([targets.columns.values.tolist()] + targets.values.tolist())


{'spreadsheetId': '1nryGNMj54kxOwJhcUGn0gqsBOGNni0yXyguBoRLzKfY',
 'updatedRange': 'google_search_results!A1:D32',
 'updatedRows': 32,
 'updatedColumns': 4,
 'updatedCells': 128}